In [ ]:
#import modules
from pytrends.request import TrendReq
from pytrendsaddon import interest_by_city
from tkinter.filedialog import askopenfilename, asksaveasfile
import pandas as pd
import geopandas as gpd
import cartoframes
from cartoframes.data.observatory import Catalog, Dataset, Variable, Enrichment
from cartoframes.viz import *
from shapely.geometry import Polygon
from sqlalchemy import create_engine
from geoalchemy2 import WKTElement
import psycopg2
from arcgis.gis import GIS
from arcgis.geocoding import geocode
from tqdm import tqdm
import folium
from ago_geocoding import ago_geocode

In [ ]:
#set weedmaps credentials
cartoframes.auth.set_default_credentials(
    username='weedmaps-admin', 
    api_key='b91fe99757c1b9142d6e206e0488ea69d4027017')

#connections to Google
pytrends_conn1 = TrendReq(hl='en-US', tz=360)
pytrends_conn2 = TrendReq(hl='en-US', tz=360)

#connect to ArcGIS Online as anonymous user
gis = GIS()

#create list of cannabis key words to pull from api
kw_list = ["Cannabis", "Marijuana", "Weed"]
pytrends_conn1.build_payload(kw_list, cat=0, timeframe = 'now 1-H', geo='US', gprop='')
pytrends_conn2.build_payload(kw_list, cat=0, timeframe = 'today 5-y', geo='US', gprop='')

#get google trends by city
canna_search_current = pytrends_conn1.interest_by_region(resolution="Region", inc_low_vol = True, inc_geo_code=False)
canna_search_old = pytrends_conn2.interest_by_region(resolution="Region", inc_low_vol = True, inc_geo_code=False)

In [ ]:
#export dataframe to csv 
canna_search_current.to_csv("canna_search_current.csv")
canna_search_old.to_csv("canna_search_old.csv")

#read it csv back in to make geoName a readable column
canna_search_current = pd.read_csv("canna_search_current.csv")
canna_search_old = pd.read_csv("canna_search_old.csv")

#rename columns
canna_search_old = canna_search_old.rename(columns={"Cannabis" : "Cannabis_old", "Marijuana" : "Marijuana_old", "Weed":"Weed_old"})
canna_search_current = canna_search_current.rename(columns={"Cannabis" : "Cannabis_current", "Marijuana" : "Marijuana_current", "Weed":"Weed_current"})

#merge dataframes based on state name
canna_search_merge = canna_search_current.merge(canna_search_old, how='inner', on = "geoName")

In [ ]:
def pct_change(df):
    #calculate percentage change columns
    df["Cannabis_Score_Pt_Change"] = round(df["Cannabis_current"] - df["Cannabis_old"])
    df["Cannabis_Score_Pct_Change"] = round(((df["Cannabis_current"] - df["Cannabis_old"])/df["Cannabis_old"]) *100)

    df["Weed_Score_Pt_Change"] = round(df["Weed_current"] - df["Weed_old"])
    df["Weed_Score_Pct_Change"] = round(((df["Weed_current"] - df["Weed_old"])/df["Weed_old"]) *100)

    df["Marijuana_Score_Pt_Change"] = round(df["Marijuana_current"] - df["Marijuana_old"])
    df["Marijuana_Score_Pct_Change"] = round(((df["Marijuana_current"] - df["Marijuana_old"])/df["Marijuana_old"]) *100)

    #format change columns
    df.style.format({"Cannabis_Score_Pt_Change":"{0:+g}"})
    df["Cannabis_Score_Pct_Change"] = df["Cannabis_Score_Pct_Change"].astype(str) + '%'

    df.style.format({"Weed_Score_Pt_Change":"{0:+g}"})
    canna_search_merge["Weed_Score_Pct_Change"] = df["Weed_Score_Pct_Change"].astype(str) + '%'

    df.style.format({"Marijuana_Score_Pt_Change":"{0:+g}"})
    df["Marijuana_Score_Pct_Change"] = df["Marijuana_Score_Pct_Change"].astype(str) + '%'
    
    return df

pct_change(canna_search_merge)

#filter out Cali
canna_search_merge = canna_search_merge[canna_search_merge.geoName != "California"]

<B>Please change the file path to where ever you save your core regions shapefile to.<B>

In [ ]:
#read in file and select main columns
usa_complete = gpd.read_file(r"/Users/dmensah/Documents/Scripts/Python/GoogleTrends/usa_ca_regions.shp")
usa_complete = usa_complete[['NAME','STUSPS','super_regi','geometry']]
usa_partial = usa_complete[usa_complete.NAME != "California"]
cali = usa_complete[usa_complete.NAME == "California"]

#set .crs attribute to web mercator projection
cali.crs = "EPSG:4326"
usa_partial.crs = "EPSG:4326"

#attribute join on states to give geometry
usa_search_merge = usa_partial.merge(canna_search_merge, how='inner', left_on = "NAME", right_on="geoName")

In [ ]:
#create list of cannabis key words to pull from api
kw_list = ["Cannabis", "Marijuana", "Weed"]
pytrends_conn1.build_payload(kw_list, cat=0, timeframe = 'now 1-H', geo='US-CA', gprop='')
pytrends_conn2.build_payload(kw_list, cat=0, timeframe = 'today 5-y', geo='US-CA', gprop='')

#get google trends by city
CA_search_current = interest_by_city(pytrends_conn1)
CA_search_old = interest_by_city(pytrends_conn2)

#get state
state = pytrends_conn1.geo
state = state.replace('US-', '')

state2 = pytrends_conn2.geo
state2 = state2.replace('US-', '')

CA_search_current["State"] = state
CA_search_old["State"] = state2

#export dataframe to csv 
CA_search_current.to_csv("CA_search_current.csv")
CA_search_old.to_csv("CA_search_old.csv")

#read it csv back in to make geoName a readable column
CA_search_current = pd.read_csv("CA_search_current.csv")
CA_search_old = pd.read_csv("CA_search_old.csv")

#create address column
CA_search_current["Address"] = CA_search_current["geoName"] + ", " + state
CA_search_old["Address"] = CA_search_old["geoName"] + ", " + state

#rename columns
CA_search_old = CA_search_old.rename(columns={"Cannabis" : "Cannabis_old", "Marijuana" : "Marijuana_old", "Weed":"Weed_old"})
CA_search_current = CA_search_current.rename(columns={"Cannabis" : "Cannabis_current", "Marijuana" : "Marijuana_current", "Weed":"Weed_current"})

#geocode and create geodataframe
CA_search_old = ago_geocode(CA_search_old, "Address")
CA_search_current = ago_geocode(CA_search_current, "Address")

In [ ]:
#spatial join on super regions
super_region_old = gpd.sjoin(CA_search_old, cali, how = "inner", op="within")
super_region_current = gpd.sjoin(CA_search_current, cali, how = "inner", op="within")

In [ ]:
#select main columns
super_region_old = super_region_old[["super_regi","Cannabis_old", "Marijuana_old", "Weed_old"]]
super_region_current = super_region_current[["super_regi","Cannabis_current", "Marijuana_current", "Weed_current"]]

#take mean of numeric columns aggregated by region
super_region_old = super_region_old.groupby("super_regi").mean()
super_region_current = super_region_current.groupby("super_regi").mean()

#merge based on super regions
super_region = super_region_current.merge(super_region_old, on="super_regi")

#convert to dataframe
super_region_df = pd.DataFrame(super_region)

#join geopandas df to pandas df
super_region_gdf = cali.merge(super_region_df, on='super_regi')

#calculate pct change
super_region_gdf = pct_change(super_region_gdf)

In [ ]:
#append geodataframes to have USA
usa_search_complete = usa_search_merge.append(super_region_gdf)

#assign cali attribute where it is current NA
usa_search_complete["geoName"] = usa_search_complete["geoName"].fillna(usa_search_complete["super_regi"])


#create index
usa_search_complete.reset_index(inplace=True)
usa_search_complete

In [ ]:
#rename score change columns of dataframe
usa_search_complete = usa_search_complete.rename(columns={"geoName":"State","Cannabis_Score_Pt_Change":"Cannabis", "Marijuana_Score_Pt_Change":"Marijuana","Weed_Score_Pt_Change":"Weed"})

#convert to json
usa_search_complete_json = usa_search_complete.to_json()
    
#convert to dataframe
usa_search_complete = pd.DataFrame(usa_search_complete)

#set up folium map
m = folium.Map(location=[48, -102], zoom_start=3, tiles= None, overlay=False)

#categories for bins
Cannabis = usa_search_complete["Cannabis"]
Marijuana = usa_search_complete["Marijuana"]
Weed = usa_search_complete["Weed"]


#create bins ITS THE BINS THROWING THE ERROR. DONT ADD NAME
bins = [list(Cannabis.quantile([0, 0.20, 0.40, 0.60, 0.80, 1])), 
        list(Marijuana.quantile([0, 0.20, 0.40, 0.60, 0.80, 1])), 
        list(Weed.quantile([0, 0.20, 0.40, 0.60, 0.80, 1]))]

#feature groups
feature_gp0 = folium.FeatureGroup(name = "Cannabis", overlay = False).add_to(m)
feature_gp1 = folium.FeatureGroup(name = "Marijuana", overlay = False).add_to(m)
feature_gp2 = folium.FeatureGroup(name = "Weed", overlay = False).add_to(m)

fs = [feature_gp0, feature_gp1, feature_gp2]

names = ["Cannabis","Marijuana", "Weed"]

In [ ]:
#chloropleth
for i, b in zip(range(len(names)), bins):
    choropleth = folium.Choropleth(
    geo_data = usa_search_complete_json,
    data = usa_search_complete,
    name = "choropleth",
    columns = ["State", names[i]],
    key_on="feature.properties.State",
    fill_color="YlGnBu",
    fill_opacity=0.6,
    line_opacity=0.5,
    legend_name="Score",
    highlight=True,
    line_color='black',
    bins = b).geojson.add_to(fs[i])

#legend
from branca.colormap import linear
colormap = linear.YlGnBu_09.to_step(5)
colormap.caption = 'Google Trends Score (Point Change)'
colormap.add_to(m)
    
#geojson for labels
geojson =  folium.GeoJson(data=usa_search_complete_json,
                          name='State',
                          smooth_factor=2,
                          style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                          highlight_function=lambda x: {'weight':3,'fillColor':'grey'}).add_to(choropleth)

folium.features.GeoJsonPopup(fields=['State',"Cannabis","Marijuana","Weed"], labels=True).add_to(geojson)

In [ ]:
#folium.TileLayer('cartodbdark_matter',overlay=True,name="dark mode").add_to(m)
folium.TileLayer('cartodbpositron',overlay=True,name="light mode").add_to(m)

folium.LayerControl(collapsed=False).add_to(m)
m.save('search_trends.html')
m